<div align='center' ><font size='70'>Allocator</font></div>

## **Allocator**

* 空间配置器接管了所有**内存**分配工作

* **SGI STL** 提供了两级分配器，**一级**分配器和**二级**分配器

* **一级**分配器直接使用**malloc**和**free**，**二级**分配器则提供了更强大的内存管理功能

```c++

#define _OOM_ "out of memory!!!"

class MallocAlloc {

public:
	inline static void* allocate(size_t n) {
		void* result = ::malloc(n);
		if (nullptr == result) throw _OOM_;
		return result;
	}

	inline static void deallocate(void* p, size_t /*pad*/) {
		::free(p);
	}

	inline static void* reallocate(void* p, size_t/*old_Sz*/, size_t new_sz) {
		void* result = ::realloc(p, new_sz);
		if (nullptr == result) throw _OOM_;
		return result;
	}
};
```

## 特殊的空间配置器 **std::alloc**

* **allocator**是基于基本内存配置/释放的**简单**封装，其没有考虑效率上的优化

* **malloc / new**效率很低？ [**[malloc.c]**](./code/malloc.c)

* [**CSAPP Malloc Lab**](https://github.com/kcxain/CSAPP-Lab/tree/master/initial_labs/08_Malloc%20Lab/traces)

* <img text-align="center" src="./src/block.png" width="70%"/>

* 第二级配置器管理内存方式与**malloc**相当**类似**


## 设计思想

* 为了减少内存碎片（**cookie**），提高内存利用率

* 第2级配置器采用**memory pool**的设计思想，管理小于**128bytes**的内存块

* **new**的实现包含了两个部分
    * **::operator new**分配内存（相当于**malloc**）
    * 调用构造函数，构造对象

* **delete**同理
    * 调用析构函数，析构对象
    * **::operator delete**释放内存（相当于**free**）

<img align="right" src="./src/memory-pool.jpg" width="45%" border="0">

## 小区块管理

* **SGI STL**将**内存池**划分为**16**个链表

* 分别管理**8**，**16**，**24**，...，**128**字节的内存块

* 16个链表的**节点**，每个节点包含一个**指针**

* 大于**128**字节的内存块，采用第一级配置器分配

<br></br><br></br>

```c
struct Obj {
	struct Obj* m_free_list_link;
};
```

## [实现细节](https://github.com/steveLauwh/SGI-STL/blob/master/SGI-STL%20V3.3/stl_alloc.h)

* 辅助函数
    * **round_up**：将**bytes**上调至**8**的倍数
    * **free_list_index**：根据**bytes**计算在**16**个链表中的位置
    * **refill**：从内存池中取空间给**free_list**使用
    * **chunk_alloc**：从内存池中取空间给**refill**使用
* 对外接口
    * **allocate**：分配内存
    * **deallocate**：释放内存
    * **reallocate**：重新分配内存

## 构造与析构

* **STL**分工精细，将**内存分配/释放**和**对象构造/析构**分开

* **allocator**只负责内存分配，**construct**负责[对象构造](https://github.com/steveLauwh/SGI-STL/blob/master/SGI-STL%20V3.3/stl_construct.h)

* **性能**——偏特化（**partial specialization**）

* [**stl_construct.h**](https://github.com/steveLauwh/SGI-STL/blob/master/SGI-STL%20V3.3/stl_construct.h)

```c++
	template<typename T1, typename T2>
	inline void Construct(T1* p, const T2& value) {
		new ((void*)p) T1(value);
	}

	template<typename T1>
	inline void Construct(T1* p) {
		new ((void*)p) T1();
	}
	// special 
	inline void Construct(int* p) {}
    // ... ...

	inline void Construct(int* p, const int value) { *p = value; }
    // ... ...

	template<typename T1, typename T2>
	inline void construct(T1* p, const T2& value) {
		Construct(p, value);
	}

	template<typename T1>
	inline void construct(T1* p) {
		Construct(p);
	}
```

<img align="right" src="./src/construct.png" width="50%" border="0">

## **Destroy** 示意图

* 根据**对象**是否具备**trivial destructor**

* **trivial destructor**，**destroy**什么都不做

* **non-trivial destructor**，调用**destructor**

* **编译器**怎么知道**destructor**是否是**trivial**？

* [**type_traits**](https://en.cppreference.com/w/cpp/header/type_traits) [**[type_traits.cpp]**](./code/type_traits.cpp)

## 内存基本处理[函数](https://github.com/steveLauwh/SGI-STL/blob/master/SGI-STL%20V3.3/stl_uninitialized.h)

* **uninitialized_copy**
    * 上层 **copy**
    * 批量复制对象

* **uninitialized_fill**
    * 上层 **fill**
    * 批量填充对象（给定起终点）

* **uninitialized_fill_n**
    * 上层 **fill_n**
    * 批量填充对象（给定数量）


<img align="right" src="./src/uninitialized_copy.png" width="60%" border="0">

## 具体实现

* **偏特化**无处不在

* 指针类型，采用**memmove()**

* is [**Plain Old Data (POD)**](https://www.educative.io/answers/what-are-plain-old-data-pod-types-in-cpp) type

    * **false_type**，构造函数

    * **true_type**，**copy()**


<div align='center'><font size='70'>End</font></div>